In [26]:
import os
import warnings
import mlflow
import pandas as pd

from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
from mlflow.store.artifact.artifact_repository_registry import get_artifact_repository

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

# exp_id = mlflow.get_experiment_by_name("Polyakov_Egor").experiment_id


exp_id = mlflow.create_experiment(name="Polyakov_Egor")
search_exp = mlflow.search_experiments(filter_string="name = 'Polyakov_Egor'")
print(f'созданный экспиремент: {search_exp}')


# dataset California
calif = fetch_california_housing(as_frame=True)

# делаем сплит данных
X_train, X_test, y_train, y_test = train_test_split(calif['data'], calif['target'])
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

scal = StandardScaler()
X_train_scal = pd.DataFrame(scal.fit_transform(X_train), columns=X_train.columns, index= X_train.index)
X_val_scal = pd.DataFrame(scal.transform(X_val), columns=X_train.columns, index= X_val.index)
X_test_scal = pd.DataFrame(scal.transform(X_test), columns=X_train.columns, index= X_test.index)


models = {
    "RandomForestRegressor": RandomForestRegressor(),
    "LinearRegression": LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor()
}

with mlflow.start_run(run_name="@Polyakov_Egor", experiment_id=exp_id, description = "parent") as run:
    for model_name in models.keys():
        with mlflow.start_run(run_name=model_name, experiment_id=exp_id, nested=True) as child_run:
            model = models[model_name]
            model.fit(X_train_scal,y_train)
            predict = model.predict(X_val_scal)
    
            df_val = X_val_scal.copy()
            df_val['target'] = y_val
    
            signature = infer_signature(X_test_scal, predict)
            model_info = mlflow.sklearn.log_model(model, 'linreg', signature=signature, registered_model_name=f"sklearn-{model_name}")
            mlflow.evaluate(
            model=model_info.model_uri,
            data=df_val,
            targets="target",
            model_type="regressor",
            evaluators=["default"],
        )
    
    
            